In [1]:
import pandas as pd
import pycountry
import unicodedata
import numpy as np

df = pd.read_csv('stocks_traded_total_value.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4080 entries, 0 to 4079
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  4080 non-null   object 
 1   Year          4080 non-null   int64  
 2   Value (USD)   4080 non-null   float64
 3   Status        4080 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 127.6+ KB


In [3]:
df.head()

,Country Name,Year,Value (USD),Status
0,Arab World,2006,1.585737e+12,Normal value
1,Arab World,2010,3.069249e+11,Normal value
2,Arab World,2011,3.792060e+11,Normal value
3,Arab World,2012,5.984638e+11,Normal value
4,Arab World,2014,8.071476e+11,Normal value


In [4]:
# transform the column header

# Mapping old names to your specific new names
df = df.rename(columns={
    'Country Name': 'countryName_',
    'Year': 'year_',
    'Value (USD)': 'value_Usd_',
    'Status': 'status_'
})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4080 entries, 0 to 4079
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   countryName_  4080 non-null   object 
 1   year_         4080 non-null   int64  
 2   value_Usd_    4080 non-null   float64
 3   status_       4080 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 127.6+ KB


In [5]:
# change dtype

# 1. year_ (int -> object)
# Convert the 'year_' column to object (string)
df['year_'] = df['year_'].astype(str)

# Verify the change
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4080 entries, 0 to 4079
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   countryName_  4080 non-null   object 
 1   year_         4080 non-null   object 
 2   value_Usd_    4080 non-null   float64
 3   status_       4080 non-null   object 
dtypes: float64(1), object(3)
memory usage: 127.6+ KB
None


In [6]:
# fix the 'value_Usd_' column value

# This tells pandas to show floats with 2 decimal places and no scientific notation
pd.options.display.float_format = '{:.2f}'.format

df.head()

,countryName_,year_,value_Usd_,status_
0,Arab World,2006,1585736810000.00,Normal value
1,Arab World,2010,306924930000.00,Normal value
2,Arab World,2011,379205990000.00,Normal value
3,Arab World,2012,598463770000.00,Normal value
4,Arab World,2014,807147637737.54,Normal value


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4080 entries, 0 to 4079
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   countryName_  4080 non-null   object 
 1   year_         4080 non-null   object 
 2   value_Usd_    4080 non-null   float64
 3   status_       4080 non-null   object 
dtypes: float64(1), object(3)
memory usage: 127.6+ KB


In [8]:
# countryName_ column = only country will be included in the dataset. Other that (

# 1. Define the exact red-flag keywords
non_country_indicators = [
    'income', 'dividend', 'total', 'world', 'area', '&', 
    'members', 'excluding', 'only', 'sub-saharan', 'europe &', 
    'asia &', 'central europe', 'arab rep', 'european union', 
    'ida blend', 'islamic rep', 'other small states', 'north america', 'south asia'
]

# 2. Re-create the list of names to drop
not_countries = [
    name for name in df['countryName_'].unique() 
    if any(word in name.lower() for word in non_country_indicators)
]

# 3. Overwrite the original 'df' with the cleaned version
# The ~ symbol means "NOT", so we keep everything NOT in the not_countries list
df = df[~df['countryName_'].isin(not_countries)].copy()

# 4. Reset index so it's a fresh start (0, 1, 2...)
df.reset_index(drop=True, inplace=True)

# 5. Check the result
print(f"New Row Count: {len(df)}")

# Now this will show 2962 rows
df.info()

New Row Count: 2887
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2887 entries, 0 to 2886
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   countryName_  2887 non-null   object 
 1   year_         2887 non-null   object 
 2   value_Usd_    2887 non-null   float64
 3   status_       2887 non-null   object 
dtypes: float64(1), object(3)
memory usage: 90.3+ KB


In [9]:
# 1. Get all unique names
all_unique = df['countryName_'].unique()

# 2. Define words that only appear in region/income groups
non_country_indicators = [
    'income', 'dividend', 'total', 'world', 'area', '&', 
    'members', 'excluding', 'only', 'sub-saharan', 'europe &', 'asia &', 'north america', 'south asia'
]

# 3. Separate them
not_countries = [
    name for name in all_unique 
    if any(word in name.lower() for word in non_country_indicators)
]

# 4. Filter them out to get ONLY real countries
real_countries = [name for name in all_unique if name not in not_countries]

print("--- These are the Aggregates (To be removed) ---")
print(not_countries)

print("\n--- These are the Actual Countries ---")
print(real_countries)

--- These are the Aggregates (To be removed) ---
[]

--- These are the Actual Countries ---
['Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Bermuda', 'Botswana', 'Brazil', 'Bulgaria', 'Canada', 'Cayman Islands', 'Channel Islands', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cyprus', 'Czechia', 'Côte d’Ivoire', 'Denmark', 'Ecuador', 'Estonia', 'Eswatini', 'Finland', 'France', 'Germany', 'Ghana', 'Greece', 'Hong Kong SAR, China', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Korea, Rep.', 'Kuwait', 'Latvia', 'Lebanon', 'Lithuania', 'Luxembourg', 'Malaysia', 'Malta', 'Mauritius', 'Mexico', 'Montenegro', 'Morocco', 'Namibia', 'Netherlands', 'New Zealand', 'Nigeria', 'Norway', 'Oman', 'Pakistan', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Qatar', 'Romania', 'Russian Federation', '

In [10]:
# rename countryNames

# 1. Function to convert specialized characters (accents/umlauts) to normal letters
def remove_accents(text):
    if not isinstance(text, str):
        return text
    # Normalize to 'NFKD' form and encode to ASCII, ignoring the specialized marks
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

# 2. Apply the transformation
df['countryName_'] = df['countryName_'].apply(remove_accents)

# 3. Check specific examples to verify
print("Transformation Check:")
print(f"Sample: {df[df['countryName_'].str.contains('Turk')]['countryName_'].unique()}")
print(f"Sample: {df[df['countryName_'].str.contains('Cote')]['countryName_'].unique()}")

Transformation Check:
Sample: ['Turkiye']
Sample: ['Cote dIvoire']


In [11]:
unique_list = df['countryName_'].unique().tolist()
print(unique_list)

['Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Bermuda', 'Botswana', 'Brazil', 'Bulgaria', 'Canada', 'Cayman Islands', 'Channel Islands', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cyprus', 'Czechia', 'Cote dIvoire', 'Denmark', 'Ecuador', 'Estonia', 'Eswatini', 'Finland', 'France', 'Germany', 'Ghana', 'Greece', 'Hong Kong SAR, China', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Korea, Rep.', 'Kuwait', 'Latvia', 'Lebanon', 'Lithuania', 'Luxembourg', 'Malaysia', 'Malta', 'Mauritius', 'Mexico', 'Montenegro', 'Morocco', 'Namibia', 'Netherlands', 'New Zealand', 'Nigeria', 'Norway', 'Oman', 'Pakistan', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Qatar', 'Romania', 'Russian Federation', 'Rwanda', 'Saudi Arabia', 'Serbia', 'Seychelles', 'Singapore', 'Slovak Republic', 'Slovenia', 

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2887 entries, 0 to 2886
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   countryName_  2887 non-null   object 
 1   year_         2887 non-null   object 
 2   value_Usd_    2887 non-null   float64
 3   status_       2887 non-null   object 
dtypes: float64(1), object(3)
memory usage: 90.3+ KB


In [13]:
# add new column 'continent_'

# 1. Define the mapping dictionary based on your specific list
continent_map = {
    # Africa
    'Botswana': 'Africa', 'Cote dIvoire': 'Africa', 'Eswatini': 'Africa', 'Ghana': 'Africa', 
    'Kenya': 'Africa', 'Mauritius': 'Africa', 'Morocco': 'Africa', 'Namibia': 'Africa', 
    'Nigeria': 'Africa', 'Rwanda': 'Africa', 'Seychelles': 'Africa', 'South Africa': 'Africa', 
    'Tanzania': 'Africa', 'Tunisia': 'Africa', 'Zambia': 'Africa', 'Zimbabwe': 'Africa',
    
    # Asia & Middle East
    'Armenia': 'Asia', 'Azerbaijan': 'Asia', 'Bahrain': 'Asia', 'Bangladesh': 'Asia', 
    'China': 'Asia', 'Hong Kong SAR, China': 'Asia', 'India': 'Asia', 'Indonesia': 'Asia', 
    'Israel': 'Asia', 'Japan': 'Asia', 'Jordan': 'Asia', 'Kazakhstan': 'Asia', 
    'Korea, Rep.': 'Asia', 'Kuwait': 'Asia', 'Lebanon': 'Asia', 'Malaysia': 'Asia', 
    'Oman': 'Asia', 'Pakistan': 'Asia', 'Philippines': 'Asia', 'Qatar': 'Asia', 
    'Saudi Arabia': 'Asia', 'Singapore': 'Asia', 'Sri Lanka': 'Asia', 'Thailand': 'Asia', 
    'United Arab Emirates': 'Asia', 'Uzbekistan': 'Asia', 'Vietnam': 'Asia', 
    'West Bank and Gaza': 'Asia', 'Turkiye': 'Asia',
    
    # Europe
    'Austria': 'Europe', 'Belarus': 'Europe', 'Belgium': 'Europe', 'Bulgaria': 'Europe', 
    'Croatia': 'Europe', 'Cyprus': 'Europe', 'Czechia': 'Europe', 'Denmark': 'Europe', 
    'Estonia': 'Europe', 'Finland': 'Europe', 'France': 'Europe', 'Germany': 'Europe', 
    'Greece': 'Europe', 'Hungary': 'Europe', 'Iceland': 'Europe', 'Ireland': 'Europe', 
    'Italy': 'Europe', 'Latvia': 'Europe', 'Lithuania': 'Europe', 'Luxembourg': 'Europe', 
    'Malta': 'Europe', 'Montenegro': 'Europe', 'Netherlands': 'Europe', 'Norway': 'Europe', 
    'Poland': 'Europe', 'Portugal': 'Europe', 'Romania': 'Europe', 'Russian Federation': 'Europe', 
    'Serbia': 'Europe', 'Slovak Republic': 'Europe', 'Slovenia': 'Europe', 'Spain': 'Europe', 
    'Sweden': 'Europe', 'Switzerland': 'Europe', 'Türkiye': 'Europe', 'Ukraine': 'Europe', 
    'United Kingdom': 'Europe', 'Channel Islands': 'Europe',
    
    # North America & Caribbean
    'Barbados': 'North America', 'Bermuda': 'North America', 'Canada': 'North America', 
    'Cayman Islands': 'North America', 'Costa Rica': 'North America', 'Jamaica': 'North America', 
    'Mexico': 'North America', 'Panama': 'North America', 'Trinidad and Tobago': 'North America', 
    'United States': 'North America',
    
    # South America
    'Argentina': 'South America', 'Brazil': 'South America', 'Chile': 'South America', 
    'Colombia': 'South America', 'Ecuador': 'South America', 'Paraguay': 'South America', 
    'Peru': 'South America', 'Uruguay': 'South America', 'Venezuela, RB': 'South America',
    
    # Oceania
    'Australia': 'Oceania', 'New Zealand': 'Oceania', 'Papua New Guinea': 'Oceania'
}

# 2. Add the 'continent_' column to your dataframe
df['continent_'] = df['countryName_'].map(continent_map)

# 3. Optional: Check if any countries were missed
missing = df[df['continent_'].isna()]['countryName_'].unique()
if len(missing) > 0:
    print(f"Countries not mapped: {missing}")
else:
    print("All countries mapped successfully!")

print(df.info())

All countries mapped successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2887 entries, 0 to 2886
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   countryName_  2887 non-null   object 
 1   year_         2887 non-null   object 
 2   value_Usd_    2887 non-null   float64
 3   status_       2887 non-null   object 
 4   continent_    2887 non-null   object 
dtypes: float64(1), object(4)
memory usage: 112.9+ KB
None


In [14]:
# Round to 2 decimal places
df['value_Usd_'] = df['value_Usd_'].round(2)

df.head()

,countryName_,year_,value_Usd_,status_,continent_
0,Argentina,1975,450000000.00,Normal value,South America
1,Argentina,1976,806000000.00,Normal value,South America
2,Argentina,1977,239000000.00,Normal value,South America
3,Argentina,1978,352000000.00,Normal value,South America
4,Argentina,1979,1471500000.00,Normal value,South America


In [17]:
# additional cleaning (remove rows that have '-' in 'value_Usd_' column)

# 1. Convert "-" to NaN (Not a Number) so Python recognizes them as missing
df['value_Usd_'] = df['value_Usd_'].replace('-', np.nan)

# 2. Remove (drop) the rows where 'Value (USD)' is now NaN
df = df.dropna(subset=['value_Usd_'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2887 entries, 0 to 2886
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   countryName_  2887 non-null   object 
 1   year_         2887 non-null   object 
 2   value_Usd_    2887 non-null   float64
 3   status_       2887 non-null   object 
 4   continent_    2887 non-null   object 
dtypes: float64(1), object(4)
memory usage: 112.9+ KB


In [19]:
# Export to CSV
# index=False prevents pandas from adding a new 'Unnamed: 0' column for the row numbers
df.to_csv('stock_market_data_cleaned.csv', index=False)

print("File exported successfully!")

File exported successfully!
